[View in Colaboratory](https://colab.research.google.com/github/youtside/colab/blob/master/Docker_&_Kube.ipynb)

# Microservices + Events + Docker
https://www.youtube.com/watch?v=sSm2dRarhPo

## Monolith vs Microservices
Microservices: functional decomposition

### Benefits
Trio: 
- Architecture
- Process: Agile, Continuous delivery, ...
- Organization: Small, autonomous, teams (6~8)


### Drawbacks
- __Complexity__ of developing a distributed system
    - Implementing inter-process communication
    - Handling partial failures
- Complexity of implementing business transactions that _span multiple databases_ (without 2PC)
- Complexity of testing a distributed system
- Complexity of deploying and operating a distributed system
- Managing the development and deployment of features that span multiple services

### Issue to address
- How to deploy the services?
- How do the services communicate? 
- How do clients of the application communicate with the services? 
- How to partition the system into services? 
- How to deal with distributed data management problems

## Event-driven Microservices
### Data management patterns
__Single shared database__ or __Database per service__

- Shared Database
![Shared Database](Asset/Microservices - Shared database.png)


- Database per service
    - each services should have its own private data that is only accessible through that service's API
    - Loose coupling but more complex
    - 2PC (aka. distributed transactions) is not viable choice for most modern applications - 2 Phase Commit / the cap the cap theorem / data consistency

### Event-driven architecture
- Whenever something significant happens (state changes), publish an event
- another service can consume this event and react accordingly, update its own data

- Use event-driven, eventually consistent order processing
    - Ebay model many years ago
![Event Driven](Asset/Microservices - Event Driven.png)

- __Problem__: How to atomically update database and publish an event?
    - Dual write problem
    - Tradition way: distributed transaction [Begin - update DB - update message broker - commit transaction]
    - but no distributed transaction in modern application
    
- Reliably publish events when state changes

### Use event-sourcing
Event centric approach to persistence. i.e., order = sequence changing of an event 
- Event table:

| Aggregate id | Aggregate type | Event id | Event type | Event Data |
|-----|-------|-----|---------------|-----|
| 101 | Order | 901 | OrderCreated  | ... |
| 101 | Order | 902 | OrderApproved | ... |
| 101 | Order | 903 | OrderShipped  | ... |

- Subscriber: Pull the event table

- Reply events to recreate state
    - Read Events to recompute the current state
    - Periodically snapshot to avoid loading all events

### Benefits of event sourcing
- Solves data consistency issues in a Microservices/NoSQL based architecture
- Reliable event publishing: publishes events needed by predictive analytics etc, user notifications, ...
- Eliminates O/R mapping problem (mostly)
- Reifies state changes:
    - Build in, reliable audit log
    - temporal queries
- Preserved history => More easily implement future requirements

### Drawbacks of event sourcing
- Requires application rewrite
- Weird and unfamiliar style of programming
- Events = a historical record of your bad design decisions
- Must handle duplicate events: idempotent handlers or duplicate detection
- Querying the event store can be challenging


### Queries?
- Find recent, valuable customers:
```SQL
SELECT * FROM CUSTOMERc, ORDER o 
WHERE c.id = o.ID AND o.ORDER_TOTAL > 100000 AND o.STATE = 'SHIPPED' AND c.CREATION_DATE > ?
```
- Microservices: No longer easy 
    - in 2 databases
    - Event Source: Writing a query involving the current state is non-trival

### Command Query Responsibility Segregation (CQRS)
![CQRS](Asset/Microservices - CQRS.png)


## Developing and deploying microservices using Docker
TBD